# 导入所需的库

In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader,ConcatDataset
import torch.nn as nn
import os
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np
import csv
import torchvision

# 参数

In [2]:
config = {
    'seed': 0,
    'train_root':'./food-11/training/labeled',
    'valid_root':'./food-11/validation',
    'test_root':'./food-11/testing',
    'unlabeled_root':'./food-11/training/unlabeled',
    'batch_size':64,
    'early_stop':10,
    'n_epochs':80,
    'learning_rate':1e-2,
    'save_path':'./models/model.ckpt',
    'do_semi':False,
    'vflip_probability':0.5,
    'hflip_probability':0.1,
}
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda



# 一些有用的函数

In [3]:
def same_seed(seed): 
    '''随机种子'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# 数据

## Tranforms
对图像进行预处理

In [4]:
train_tfm = transforms.Compose([   
    transforms.Resize((128,128)),
#     transforms.RandomResizedCrop((128, 128), scale=(0.7, 1.0)),
#     transforms.RandomHorizontalFlip(0.5),
#     transforms.RandomVerticalFlip(0.5),
#     transforms.RandomRotation(180),
#     transforms.RandomAffine(30),

#     transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
])

test_tfm = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

## Dataset

In [5]:
#train_set = DatasetFolder(config['train_root'],transform = train_tfm,extensions="jpg",loader = lambda x: Image.open(x))
#valid_set = DatasetFolder(config['valid_root'],transform = train_tfm,extensions="jpg",loader = lambda x: Image.open(x))
test_set = DatasetFolder(config['test_root'],transform = test_tfm,extensions="jpg",loader = lambda x: Image.open(x))
unlabeled_set = DatasetFolder(config['unlabeled_root'],transform = train_tfm,extensions="jpg",loader = lambda x: Image.open(x))

In [6]:
def get_pseudo_labels(dataset, model, threshold=0.65):
    data_loader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=False)
    model.eval()
    softmax = nn.Softmax(dim=-1)
    for imgs,labels in data_loader:
        with torch.no_grad():
            logits = model(imgs.to(device))
        probs = softmax(logits)
    model.train()
    return dataset

## DataLoader

In [7]:
#train_loader = DataLoader(train_set,batch_size = config['batch_size'],shuffle = True,pin_memory = True,num_workers = 0)
#valid_loader = DataLoader(valid_set,batch_size = config['batch_size'],shuffle = True,pin_memory = True,num_workers = 0)
test_loader = DataLoader(test_set,batch_size = config['batch_size'],shuffle = False)

# Neural Network Model

In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = self.relu(out)
        
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block=ResidualBlock, out_channels=64, num_blocks=2, stride=1)
        self.layer2 = self._make_layer(block=ResidualBlock, out_channels=128, num_blocks=2, stride=2)
        self.layer3 = self._make_layer(block=ResidualBlock, out_channels=256, num_blocks=2, stride=2)
        self.layer4 = self._make_layer(block=ResidualBlock, out_channels=512, num_blocks=2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_features=512, out_features=11)
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_features=out_channels)
            )
        
        layers = []
        layers.append(block(in_channels=self.in_channels, out_channels=out_channels, stride=stride, downsample=downsample))
        self.in_channels = out_channels
        
        for i in range(1, num_blocks):
            layers.append(block(in_channels=out_channels, out_channels=out_channels))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        
        return out

# class Image_Classification(nn.Module):
#     def __init__(self):
#         super(Image_Classification,self).__init__()
#         self.cnn_layers = nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1, 1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),

#             nn.Conv2d(64, 128, 3, 1, 1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),

#             nn.Conv2d(128, 256, 3, 1, 1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
#             nn.MaxPool2d(4, 4, 0),
#         )
#         self.fc_layers = nn.Sequential(
#             nn.Linear(256 * 8 * 8, 256),
#             nn.ReLU(),
#             nn.Dropout(p=0.5, inplace=False),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Dropout(p=0.5, inplace=False),
#             nn.Linear(256, 11)
#         )
    
#     def forward(self,x):
#         x = self.cnn_layers(x)
#         x = x.flatten(1)
#         x = self.fc_layers(x)
        
#         return x

# Training Loop

In [9]:
def trainer(do_semi,model,config,device):   
    same_seed(config['seed'])
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
    
    if not os.path.isdir('./models'):
        os.mkdir('./models')
        
    train_set = DatasetFolder(config['train_root'],transform = train_tfm,extensions="jpg",loader = lambda x: Image.open(x))
    train_set_new = DatasetFolder(config['train_root'],transform = test_tfm,extensions="jpg",loader = lambda x: Image.open(x))
    train_set = ConcatDataset([train_set,train_set_new])
    valid_set = DatasetFolder(config['valid_root'],transform = train_tfm,extensions="jpg",loader = lambda x: Image.open(x))
    
    if do_semi:
        pseudo_set = get_pseudo_labels(unlabeled_set, model)
        train_set = ConcatDataset([train_set, pseudo_set])
        
    n_epochs,best_acc,early_stop,early_stop_count = config['n_epochs'],0,config['n_epochs'],0
    for epoch in range(n_epochs):  
        train_loader = DataLoader(train_set,batch_size = config['batch_size'],shuffle = True,pin_memory = True,num_workers = 0)
        valid_loader = DataLoader(valid_set,batch_size = config['batch_size'],shuffle = True,pin_memory = True,num_workers = 0)
        
        model.train()
        train_loss_record = []
        train_acc_record = []
        train_pbar = tqdm(train_loader)
        for imgs,labels in train_pbar:
            optimizer.zero_grad()
            imgs,labels = imgs.to(device),labels.to(device)
            logits = model(imgs)
            loss = criterion(logits,labels)
            loss.backward()
            
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            
            optimizer.step()
            acc = (logits.argmax(dim=-1)==labels).float().mean()
            #print(loss.detach().item())
            train_loss_record.append(loss.detach().item())
            train_acc_record.append(acc.item())
            
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'train_loss':loss.item(),'train_acc':acc.item()})
        train_loss_mean = sum(train_loss_record)/len(train_loss_record)
        train_acc_mean = sum(train_acc_record)/len(train_acc_record)
        
        model.eval()
        valid_loss_record = []
        valid_acc_record = []
        with torch.no_grad():
            for imgs,labels in valid_loader:
                imgs,labels = imgs.to(device),labels.to(device)
                logits = model(imgs)
                loss = criterion(logits,labels)
                acc = (logits.argmax(dim = -1) == labels).float().mean()
                valid_loss_record.append(loss.detach().item())
                valid_acc_record.append(acc.item())
        valid_loss_mean = sum(valid_loss_record)/len(valid_loss_record)
        valid_acc_mean = sum(valid_acc_record)/len(valid_acc_record)
        print(f'train_acc:{round(train_acc_mean,3)},train_loss:{round(train_loss_mean,3)},valid_acc:{round(valid_acc_mean,3)},valid_loss:{round(valid_loss_mean,3)}')
        if valid_acc_mean > best_acc:
            best_acc = valid_acc_mean
            torch.save(model.state_dict(),config['save_path'])
            print('saving model with acc {}'.format(best_acc))
            early_stop_count = 0
        else:
            early_stop_count += 1
        if early_stop_count == config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return best_acc
    return best_acc

# Start training!

In [10]:
model = ResNet18().to(device)
#model = torchvision.models.resnet18(pretrained=False).to(device)
best_acc = trainer(config['do_semi'],model,config,device)
if best_acc> 0.8:
    config['do_semi'] = True
    trainer(config['do_semi'],model,config,device)

Epoch [1/80]: 100%|██████████████████████████████████| 97/97 [00:53<00:00,  1.80it/s, train_loss=1.95, train_acc=0.375]


train_acc:0.312,train_loss:1.987,valid_acc:0.313,valid_loss:2.214
saving model with acc 0.3130681812763214


Epoch [2/80]: 100%|██████████████████████████████████| 97/97 [00:46<00:00,  2.10it/s, train_loss=1.46, train_acc=0.625]


train_acc:0.419,train_loss:1.688,valid_acc:0.393,valid_loss:1.842
saving model with acc 0.3928977278145877


Epoch [3/80]: 100%|███████████████████████████████████| 97/97 [00:47<00:00,  2.06it/s, train_loss=1.8, train_acc=0.438]


...

Epoch [34/80]: 100%|████████████████████████████████████| 97/97 [00:52<00:00,  1.84it/s, train_loss=0.023, train_acc=1]


train_acc:1.0,train_loss:0.001,valid_acc:0.597,valid_loss:1.814


Epoch [35/80]: 100%|███████████████████████████████████| 97/97 [00:53<00:00,  1.83it/s, train_loss=0.0427, train_acc=1]


train_acc:1.0,train_loss:0.002,valid_acc:0.588,valid_loss:1.874

Model is not improving, so we halt the training session.


NameError: name 'best_acc' is not defined

# predict
## predict函数

In [ ]:
def predict(test_loader,model,device):
    model.eval()
    preds = []
    for imgs,labels in tqdm(test_loader):
        imgs = imgs.to(device)
        with torch.no_grad():
            logits = model(imgs)
        pred = logits.argmax(dim=-1)
        for i in pred.cpu().numpy():
            preds.append(i)
    return preds

## save_pred

In [ ]:
def save_pred(preds,file):
    with open(file,'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id','class'])
        for i,p in enumerate(preds):
            writer.writerow([i,p])

In [ ]:
model = Image_Classification().to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader,model,device)
save_pred(preds,'pred.csv')